In [1]:
import pandas as pd
import json
import requests
import urllib
import datetime as dt
import pandas_datareader as pdr

In [2]:
security_breaches = pd.read_csv('/Users/garimajain/Desktop/SecurityBreaches-7.csv')

In [3]:
df1 = security_breaches.iloc[1:]

In [4]:
breach_data = df1.to_csv("Lab3")

In [5]:
breaches = pd.read_csv('/Users/garimajain/Desktop/Lab3.csv')

In [6]:
def get_symbol(company_name):
    url = "http://d.yimg.com/autoc.finance.yahoo.com/autoc?query={}&region=1&lang=en".format(company_name)

    result = requests.get(url).json()

    for x in result['ResultSet']['Result']:
            return x['symbol']
        
ticker = []
company = breaches.Entity
for  symbol in company:
    ticker.append(get_symbol(symbol))

In [7]:
ticker_df = pd.DataFrame(ticker)
new_data = breaches.assign(Symbol = ticker)

In [8]:
data_breach = new_data.dropna(axis=0,how='any')

In [9]:
new_data.to_csv('Lab3_with_ticker.csv')

In [10]:
start_date = dt.datetime(2004,1,1)
end_date = dt.datetime(2017,6,6)
data_source = 'yahoo'
panel_data = pdr.DataReader(ticker, data_source, start_date, end_date)

/Users/garimajain/anaconda/lib/python2.7/site-packages/pandas_datareader/yahoo/daily.py:136: SymbolWarning: Failed to read symbol: u'AOL', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/garimajain/anaconda/lib/python2.7/site-packages/pandas_datareader/yahoo/daily.py:136: SymbolWarning: Failed to read symbol: None, replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/garimajain/anaconda/lib/python2.7/site-packages/pandas_datareader/yahoo/daily.py:136: SymbolWarning: Failed to read symbol: u'CIT-U.TI', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/garimajain/anaconda/lib/python2.7/site-packages/pandas_datareader/yahoo/daily.py:136: SymbolWarning: Failed to read symbol: u'^UMIAMIFL', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/garimajain/anaconda/lib/python2.7/site-packages/pandas_datareader/yahoo/daily.py:136: SymbolWarning: Failed to read symbol: u'JPM-PB', replacing with NaN.
 

In [11]:
panel_data

<class 'pandas.core.panel.Panel'>
Dimensions: 6 (items) x 3669 (major_axis) x 64 (minor_axis)
Items axis: Adj Close to Volume
Major_axis axis: 2004-01-01 00:00:00 to 2017-06-07 00:00:00
Minor_axis axis: None to ^UMIAMIFL

In [12]:
close = panel_data.ix['Close']

In [13]:
all_weekdays = pd.date_range(start=start_date, end=end_date, freq='B')

In [14]:
close = close.reindex(all_weekdays)

In [15]:
revised_data = close.dropna(axis=1,how='all')

In [16]:
avg_by_week = revised_data.groupby(pd.TimeGrouper(freq='1W')).mean()

In [22]:
avg_by_week.head()

,108790.KQ,600488.SS,AABA,AAPL,ADBE,ADP,AMTD,ANTM,AT,ATGYX,...,TALK.L,TER.AX,TGT,TWTR,UBI.PA,UPS,VOD,VTKLF,YAHOY,^CEX
2004-01-04,NaN,1.893455,22.700001,1.520000,19.549999,31.690842,14.1500,38.139999,NaN,25.430000,...,NaN,NaN,37.720001,NaN,5.60125,73.650002,29.485947,NaN,NaN,197.270004
2004-01-11,NaN,1.940418,23.851000,1.617285,19.420000,32.594934,14.9300,37.010000,NaN,25.622000,...,NaN,NaN,37.810000,NaN,5.72250,73.033999,31.284696,NaN,NaN,198.176001
2004-01-18,NaN,2.144392,24.313000,1.680286,18.845000,32.285639,15.2100,37.772999,NaN,25.774000,...,NaN,NaN,38.428000,NaN,5.59750,72.386000,31.648172,NaN,NaN,196.204004
2004-01-25,NaN,2.246380,23.663750,1.608572,19.033750,32.904229,16.7025,39.487499,NaN,26.210001,...,NaN,NaN,38.394999,NaN,5.56150,72.252500,31.457696,NaN,NaN,192.407497
2004-02-01,NaN,2.256040,23.446000,1.626286,18.700000,33.713154,16.1560,40.113000,NaN,26.028000,...,NaN,NaN,38.150000,NaN,5.15700,72.414000,30.129023,NaN,NaN,191.584002


In [23]:
data_rearrangemet = avg_by_week.T

In [25]:
#created a dataframe for de-normalized data about Ticker symbol, date and corresponding stock price
final_data = pd.DataFrame(columns=['Symbol','Date','Price'])
d =0
for i in range(len(data_rearrangemet)) : 
    ts = data_rearrangemet.index[i]
    for j in range(len(data_rearrangemet.iloc[i])): 
        timet = pd.to_datetime(str(data_rearrangemet.columns.values[j]))
        date = timet.strftime('%Y-%m-%d')
        final_data.loc[d]=( ts,  date , data_rearrangemet.iloc[i][j])
        d=d+1